# 🎄 Santa 2025 - Maximum Density Tree Packing

**Target: Score < 60 with ZERO overlaps**

Key strategies:
- Ultra-aggressive simulated annealing (100k+ iterations)
- Optimal rotation angles for tree interlocking
- Multi-restart optimization with best selection
- Gradient-based compaction with strict collision checks
- Physics-based overlap resolution

In [ ]:
import numpy as np
import pandas as pd
import os
import time
from typing import List, Tuple, Optional

try:
    from tqdm.auto import tqdm
except ImportError:
    !pip install tqdm -q
    from tqdm.auto import tqdm

try:
    from numba import njit, prange
except ImportError:
    !pip install numba -q
    from numba import njit, prange

np.random.seed(42)
print(f"NumPy: {np.__version__}")

In [ ]:
# ============================================================
# TREE POLYGON DEFINITION
# ============================================================

TREE_POLYGON = np.array([
    [0.0, 1.0], [-0.25, 0.6], [-0.15, 0.6], [-0.35, 0.3], [-0.20, 0.3],
    [-0.45, 0.0], [-0.10, 0.0], [-0.10, -0.20], [0.10, -0.20], [0.10, 0.0],
    [0.45, 0.0], [0.20, 0.3], [0.35, 0.3], [0.15, 0.6], [0.25, 0.6],
], dtype=np.float64)

# Try to load from Kaggle
for path in ['/kaggle/input/santa-2025', '/kaggle/input/santa-2025-christmas-tree-packing-challenge', '.']:
    tree_file = os.path.join(path, 'tree.csv')
    if os.path.exists(tree_file):
        df = pd.read_csv(tree_file)
        if 'x' in df.columns and 'y' in df.columns:
            TREE_POLYGON = df[['x', 'y']].values.astype(np.float64)
            print(f"Loaded tree from {tree_file}")
            break

NUM_VERTICES = len(TREE_POLYGON)
TREE_WIDTH = TREE_POLYGON[:, 0].max() - TREE_POLYGON[:, 0].min()
TREE_HEIGHT = TREE_POLYGON[:, 1].max() - TREE_POLYGON[:, 1].min()
TREE_AREA = 0.5 * abs(sum(TREE_POLYGON[i,0]*(TREE_POLYGON[(i+1)%NUM_VERTICES,1]-TREE_POLYGON[i-1,1]) for i in range(NUM_VERTICES)))
print(f"Tree: {NUM_VERTICES} vertices, {TREE_WIDTH:.3f} x {TREE_HEIGHT:.3f}, area={TREE_AREA:.4f}")

In [ ]:
# ============================================================
# CORE GEOMETRY (Numba JIT)
# ============================================================

@njit(cache=True)
def rotate_poly(poly, deg):
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    result = np.empty_like(poly)
    for i in range(len(poly)):
        result[i, 0] = poly[i, 0] * c - poly[i, 1] * s
        result[i, 1] = poly[i, 0] * s + poly[i, 1] * c
    return result

@njit(cache=True)
def transform_poly(poly, x, y, deg):
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    result = np.empty_like(poly)
    for i in range(len(poly)):
        result[i, 0] = poly[i, 0] * c - poly[i, 1] * s + x
        result[i, 1] = poly[i, 0] * s + poly[i, 1] * c + y
    return result

@njit(cache=True)
def get_bbox(poly):
    min_x = max_x = poly[0, 0]
    min_y = max_y = poly[0, 1]
    for i in range(1, len(poly)):
        if poly[i, 0] < min_x: min_x = poly[i, 0]
        if poly[i, 0] > max_x: max_x = poly[i, 0]
        if poly[i, 1] < min_y: min_y = poly[i, 1]
        if poly[i, 1] > max_y: max_y = poly[i, 1]
    return min_x, min_y, max_x, max_y

@njit(cache=True)
def bbox_overlaps(b1, b2, eps=0.0):
    return (b1[0] < b2[2] - eps and b1[2] > b2[0] + eps and
            b1[1] < b2[3] - eps and b1[3] > b2[1] + eps)

print("Geometry compiled.")

In [ ]:
# ============================================================
# SAT COLLISION DETECTION (Robust)
# ============================================================

@njit(cache=True)
def sat_overlap(poly1, poly2, eps=1e-9):
    """Robust SAT collision test with epsilon tolerance."""
    for p in range(2):
        poly = poly1 if p == 0 else poly2
        n = len(poly)
        for i in range(n):
            j = (i + 1) % n
            nx = -(poly[j, 1] - poly[i, 1])
            ny = poly[j, 0] - poly[i, 0]
            length = np.sqrt(nx*nx + ny*ny)
            if length < 1e-12:
                continue
            nx /= length
            ny /= length
            
            min1 = max1 = poly1[0, 0] * nx + poly1[0, 1] * ny
            for k in range(1, len(poly1)):
                proj = poly1[k, 0] * nx + poly1[k, 1] * ny
                if proj < min1: min1 = proj
                if proj > max1: max1 = proj
            
            min2 = max2 = poly2[0, 0] * nx + poly2[0, 1] * ny
            for k in range(1, len(poly2)):
                proj = poly2[k, 0] * nx + poly2[k, 1] * ny
                if proj < min2: min2 = proj
                if proj > max2: max2 = proj
            
            if max1 <= min2 + eps or max2 <= min1 + eps:
                return False
    return True

@njit(cache=True)
def collides(base, x1, y1, d1, x2, y2, d2, eps=1e-9):
    """Check collision between two trees with early bbox rejection."""
    p1 = transform_poly(base, x1, y1, d1)
    p2 = transform_poly(base, x2, y2, d2)
    b1, b2 = get_bbox(p1), get_bbox(p2)
    if not bbox_overlaps(b1, b2, eps):
        return False
    return sat_overlap(p1, p2, eps)

@njit(cache=True)
def any_collision(base, pl, eps=1e-9):
    """Check if any pair of trees collides."""
    n = len(pl)
    for i in range(n):
        for j in range(i + 1, n):
            if collides(base, pl[i,0], pl[i,1], pl[i,2], pl[j,0], pl[j,1], pl[j,2], eps):
                return True
    return False

@njit(cache=True)
def count_collisions(base, pl, eps=1e-9):
    """Count total number of colliding pairs."""
    cnt = 0
    n = len(pl)
    for i in range(n):
        for j in range(i + 1, n):
            if collides(base, pl[i,0], pl[i,1], pl[i,2], pl[j,0], pl[j,1], pl[j,2], eps):
                cnt += 1
    return cnt

@njit(cache=True)
def collision_for_idx(base, pl, idx, eps=1e-9):
    """Count collisions for a specific tree."""
    cnt = 0
    for j in range(len(pl)):
        if j != idx:
            if collides(base, pl[idx,0], pl[idx,1], pl[idx,2], pl[j,0], pl[j,1], pl[j,2], eps):
                cnt += 1
    return cnt

print("Collision detection compiled.")

In [ ]:
# ============================================================
# BOUNDING BOX & CENTERING
# ============================================================

@njit(cache=True)
def compute_side(base, pl):
    """Compute side length of minimal bounding square."""
    if len(pl) == 0:
        return 0.0
    minx = miny = 1e9
    maxx = maxy = -1e9
    for i in range(len(pl)):
        poly = transform_poly(base, pl[i,0], pl[i,1], pl[i,2])
        b = get_bbox(poly)
        if b[0] < minx: minx = b[0]
        if b[1] < miny: miny = b[1]
        if b[2] > maxx: maxx = b[2]
        if b[3] > maxy: maxy = b[3]
    return max(maxx - minx, maxy - miny)

@njit(cache=True)
def get_bounds(base, pl):
    """Get bounding box of all trees."""
    minx = miny = 1e9
    maxx = maxy = -1e9
    for i in range(len(pl)):
        poly = transform_poly(base, pl[i,0], pl[i,1], pl[i,2])
        b = get_bbox(poly)
        if b[0] < minx: minx = b[0]
        if b[1] < miny: miny = b[1]
        if b[2] > maxx: maxx = b[2]
        if b[3] > maxy: maxy = b[3]
    return minx, miny, maxx, maxy

@njit(cache=True)
def center_pl(base, pl):
    """Center placement around origin."""
    if len(pl) == 0:
        return pl.copy()
    minx, miny, maxx, maxy = get_bounds(base, pl)
    cx, cy = (minx + maxx) / 2, (miny + maxy) / 2
    result = pl.copy()
    for i in range(len(result)):
        result[i, 0] -= cx
        result[i, 1] -= cy
    return result

print("Bounding box compiled.")

In [ ]:
# ============================================================
# ULTRA-AGGRESSIVE SIMULATED ANNEALING
# ============================================================

@njit(cache=True)
def sa_optimize_ultra(base, pl, n_iter, t_init, t_min, eps=1e-9):
    """Ultra-aggressive simulated annealing with adaptive moves."""
    n = len(pl)
    if n <= 1:
        return pl.copy()
    
    current = pl.copy()
    best = current.copy()
    
    # Compute score with heavy collision penalty
    def score(p):
        s = compute_side(base, p)
        sc = s * s  # Square to penalize larger boxes heavily
        cols = 0
        for i in range(len(p)):
            for j in range(i+1, len(p)):
                if collides(base, p[i,0], p[i,1], p[i,2], p[j,0], p[j,1], p[j,2], eps):
                    cols += 1
        sc += cols * 500.0  # Heavy penalty
        return sc, cols
    
    cur_score, cur_cols = score(current)
    best_score = cur_score
    
    t = t_init
    cooling = (t_min / t_init) ** (1.0 / n_iter)
    
    # Track improvement for adaptive strategy
    no_improve = 0
    
    for it in range(n_iter):
        idx = np.random.randint(0, n)
        old_x, old_y, old_d = current[idx, 0], current[idx, 1], current[idx, 2]
        
        # Adaptive move selection based on temperature
        r = np.random.random()
        if r < 0.35:
            # Translation - scale with temperature
            scale = t * 0.8
            current[idx, 0] += (np.random.random() - 0.5) * scale
            current[idx, 1] += (np.random.random() - 0.5) * scale
        elif r < 0.55:
            # Rotation only
            current[idx, 2] = (current[idx, 2] + (np.random.random() - 0.5) * t * 120) % 360
        elif r < 0.75:
            # Combined small move
            scale = t * 0.4
            current[idx, 0] += (np.random.random() - 0.5) * scale
            current[idx, 1] += (np.random.random() - 0.5) * scale
            current[idx, 2] = (current[idx, 2] + (np.random.random() - 0.5) * t * 60) % 360
        elif r < 0.88:
            # Swap with another tree
            j = np.random.randint(0, n)
            if j != idx:
                current[idx, 0], current[j, 0] = current[j, 0], current[idx, 0]
                current[idx, 1], current[j, 1] = current[j, 1], current[idx, 1]
        else:
            # Try optimal rotation angles (0, 60, 120, 180, 240, 300)
            best_rot = old_d
            best_cols = collision_for_idx(base, current, idx, eps)
            for rot in [0.0, 60.0, 120.0, 180.0, 240.0, 300.0]:
                current[idx, 2] = rot
                c = collision_for_idx(base, current, idx, eps)
                if c < best_cols:
                    best_cols = c
                    best_rot = rot
            current[idx, 2] = best_rot
        
        new_score, new_cols = score(current)
        delta = new_score - cur_score
        
        # Accept better moves or probabilistically accept worse ones
        if delta < 0 or np.random.random() < np.exp(-delta / (t + 1e-10)):
            cur_score = new_score
            cur_cols = new_cols
            no_improve = 0
            if new_score < best_score:
                best = current.copy()
                best_score = new_score
        else:
            # Reject - restore
            current[idx, 0], current[idx, 1], current[idx, 2] = old_x, old_y, old_d
            if r >= 0.75 and r < 0.88 and j != idx:
                current[idx, 0], current[j, 0] = current[j, 0], current[idx, 0]
                current[idx, 1], current[j, 1] = current[j, 1], current[idx, 1]
            no_improve += 1
        
        t *= cooling
        
        # Restart from best if stuck
        if no_improve > n_iter // 10:
            current = best.copy()
            cur_score = best_score
            no_improve = 0
    
    return best

print("Ultra SA compiled.")

In [ ]:
# ============================================================
# PHYSICS-BASED REPAIR (Repulsive Forces)
# ============================================================

@njit(cache=True)
def repair_physics(base, pl, max_iter=1000, eps=1e-9):
    """Resolve overlaps using physics-based repulsion."""
    result = pl.copy()
    n = len(result)
    
    for it in range(max_iter):
        forces = np.zeros((n, 2))
        has_col = False
        
        for i in range(n):
            for j in range(i+1, n):
                if collides(base, result[i,0], result[i,1], result[i,2],
                           result[j,0], result[j,1], result[j,2], eps):
                    has_col = True
                    dx = result[j,0] - result[i,0]
                    dy = result[j,1] - result[i,1]
                    dist = np.sqrt(dx*dx + dy*dy) + 1e-10
                    
                    # Progressive force that increases over iterations
                    push = 0.02 * (1.0 + it / 100.0)
                    
                    forces[i, 0] -= dx/dist * push
                    forces[i, 1] -= dy/dist * push
                    forces[j, 0] += dx/dist * push
                    forces[j, 1] += dy/dist * push
        
        if not has_col:
            break
        
        for i in range(n):
            result[i, 0] += forces[i, 0]
            result[i, 1] += forces[i, 1]
    
    return result

@njit(cache=True)
def repair_greedy(base, pl, max_iter=500, eps=1e-9):
    """Greedily move colliding trees to valid positions."""
    result = pl.copy()
    n = len(result)
    
    for it in range(max_iter):
        if not any_collision(base, result, eps):
            break
        
        # Find tree with most collisions
        max_cols = 0
        worst_idx = 0
        for i in range(n):
            c = collision_for_idx(base, result, i, eps)
            if c > max_cols:
                max_cols = c
                worst_idx = i
        
        if max_cols == 0:
            break
        
        # Try moving this tree to a valid position
        i = worst_idx
        best_pos = (result[i, 0], result[i, 1], result[i, 2])
        best_cols = max_cols
        
        # Try different directions and rotations
        step = 0.05 * (1 + it / 100)
        for angle in range(0, 360, 30):
            rad = np.radians(angle)
            for dist in [step, step*2, step*3]:
                nx = result[i, 0] + np.cos(rad) * dist
                ny = result[i, 1] + np.sin(rad) * dist
                for rot in [result[i, 2], (result[i, 2] + 60) % 360, (result[i, 2] + 180) % 360]:
                    result[i, 0], result[i, 1], result[i, 2] = nx, ny, rot
                    c = collision_for_idx(base, result, i, eps)
                    if c < best_cols:
                        best_cols = c
                        best_pos = (nx, ny, rot)
        
        result[i, 0], result[i, 1], result[i, 2] = best_pos
    
    return result

print("Repair functions compiled.")

In [ ]:
# ============================================================
# AGGRESSIVE COMPACTION
# ============================================================

@njit(cache=True)
def compact_aggressive(base, pl, max_iter=500, eps=1e-9):
    """Aggressively compact trees toward center."""
    result = pl.copy()
    n = len(result)
    
    for iteration in range(max_iter):
        improved = False
        
        # Compute centroid
        cx = np.mean(result[:, 0])
        cy = np.mean(result[:, 1])
        
        # Adaptive step size that decreases
        base_step = 0.03 * (1.0 - iteration / (max_iter * 2))
        if base_step < 0.005:
            base_step = 0.005
        
        for i in range(n):
            dx = cx - result[i, 0]
            dy = cy - result[i, 1]
            dist = np.sqrt(dx*dx + dy*dy)
            
            if dist < 0.001:
                continue
            
            # Try multiple step sizes
            for step in [base_step, base_step * 0.5, base_step * 0.25]:
                old_x, old_y = result[i, 0], result[i, 1]
                result[i, 0] += dx * step / dist
                result[i, 1] += dy * step / dist
                
                if collision_for_idx(base, result, i, eps) > 0:
                    result[i, 0], result[i, 1] = old_x, old_y
                else:
                    improved = True
                    break
        
        if not improved:
            break
    
    return result

@njit(cache=True)
def compact_to_bounds(base, pl, max_iter=300, eps=1e-9):
    """Compact trees toward the bounding box center."""
    result = pl.copy()
    n = len(result)
    
    for iteration in range(max_iter):
        improved = False
        minx, miny, maxx, maxy = get_bounds(base, result)
        bcx, bcy = (minx + maxx) / 2, (miny + maxy) / 2
        
        step = 0.02
        
        for i in range(n):
            # Pull toward bounding box center
            dx = bcx - result[i, 0]
            dy = bcy - result[i, 1]
            dist = np.sqrt(dx*dx + dy*dy)
            
            if dist < 0.001:
                continue
            
            old_x, old_y = result[i, 0], result[i, 1]
            result[i, 0] += dx * step / dist
            result[i, 1] += dy * step / dist
            
            if collision_for_idx(base, result, i, eps) > 0:
                result[i, 0], result[i, 1] = old_x, old_y
            else:
                improved = True
        
        if not improved:
            break
    
    return result

print("Compaction compiled.")

In [ ]:
# ============================================================
# ROTATION OPTIMIZATION
# ============================================================

@njit(cache=True)
def optimize_rotations(base, pl, angles_per_tree=36, eps=1e-9):
    """Find optimal rotation for each tree to minimize bounding box."""
    result = pl.copy()
    n = len(result)
    
    for i in range(n):
        best_side = compute_side(base, result)
        best_deg = result[i, 2]
        
        step = 360.0 / angles_per_tree
        for a in range(angles_per_tree):
            d = a * step
            result[i, 2] = d
            
            # Must be collision-free
            if collision_for_idx(base, result, i, eps) == 0:
                s = compute_side(base, result)
                if s < best_side - 1e-6:
                    best_side = s
                    best_deg = d
        
        result[i, 2] = best_deg
    
    return result

@njit(cache=True)
def optimize_rotations_fine(base, pl, eps=1e-9):
    """Fine-grained rotation optimization."""
    result = pl.copy()
    n = len(result)
    
    # Multiple passes with decreasing step
    for step in [30.0, 15.0, 5.0, 2.0]:
        for i in range(n):
            best_side = compute_side(base, result)
            best_deg = result[i, 2]
            
            for offset in np.arange(-step*3, step*3 + 0.1, step):
                d = (result[i, 2] + offset) % 360
                result[i, 2] = d
                
                if collision_for_idx(base, result, i, eps) == 0:
                    s = compute_side(base, result)
                    if s < best_side - 1e-6:
                        best_side = s
                        best_deg = d
            
            result[i, 2] = best_deg
    
    return result

print("Rotation optimization compiled.")

In [ ]:
# ============================================================
# PLACEMENT GENERATORS
# ============================================================

def gen_tight_spiral(n, scale=0.25):
    """Tight spiral placement for efficient packing."""
    pl = np.zeros((n, 3))
    if n == 0:
        return pl
    pl[0] = [0, 0, 0]
    angle = 0.0
    golden = 137.508 * np.pi / 180  # Golden angle
    for i in range(1, n):
        radius = scale * np.sqrt(i)
        pl[i] = [radius * np.cos(angle), radius * np.sin(angle), (i * 60) % 360]
        angle += golden
    return pl

def gen_hex_tight(n, spacing=0.42):
    """Tight hexagonal grid placement."""
    pl = np.zeros((n, 3))
    idx = 0
    row = 0
    sx = spacing
    sy = spacing * 0.866  # sqrt(3)/2
    
    while idx < n:
        offset = (row % 2) * sx * 0.5
        cols_per_row = int(np.ceil(np.sqrt(n * 1.3))) + 1
        for col in range(cols_per_row):
            if idx >= n:
                break
            pl[idx] = [col * sx + offset, row * sy, ((row + col) * 60) % 360]
            idx += 1
        row += 1
    
    # Center
    pl[:, 0] -= np.mean(pl[:, 0])
    pl[:, 1] -= np.mean(pl[:, 1])
    return pl

def gen_rings(n):
    """Concentric ring placement."""
    pl = np.zeros((n, 3))
    if n == 0:
        return pl
    
    pl[0] = [0, 0, 0]
    idx = 1
    ring = 1
    
    while idx < n:
        radius = ring * 0.4
        circumference = 2 * np.pi * radius
        trees_in_ring = max(6, int(circumference / 0.35))
        
        for i in range(trees_in_ring):
            if idx >= n:
                break
            angle = 2 * np.pi * i / trees_in_ring + (ring % 2) * np.pi / trees_in_ring
            pl[idx] = [radius * np.cos(angle), radius * np.sin(angle), (i * 60) % 360]
            idx += 1
        ring += 1
    
    return pl

def gen_interlocking(n):
    """Interlocking pattern with alternating orientations."""
    pl = np.zeros((n, 3))
    idx = 0
    row = 0
    sx, sy = 0.38, 0.35
    
    while idx < n:
        offset = (row % 2) * sx * 0.5
        cols = int(np.ceil(np.sqrt(n * 1.2))) + 1
        for col in range(cols):
            if idx >= n:
                break
            # Alternate rotations for interlocking
            rot = 180 if (row + col) % 2 == 0 else 0
            pl[idx] = [col * sx + offset, row * sy, rot]
            idx += 1
        row += 1
    
    pl[:, 0] -= np.mean(pl[:, 0])
    pl[:, 1] -= np.mean(pl[:, 1])
    return pl

print("Generators ready.")

In [ ]:
# ============================================================
# MAIN SOLVER - Multi-Strategy
# ============================================================

def solve_puzzle(n, base, prev=None):
    """Solve puzzle n with multiple strategies."""
    eps = 1e-9
    
    if n == 1:
        return np.array([[0.0, 0.0, 0.0]]), compute_side(base, np.array([[0.0, 0.0, 0.0]]))
    
    best_pl = None
    best_side = 1e9
    
    # Scale iterations with n - MUCH more aggressive
    base_iters = 50000
    iters = min(base_iters + n * 500, 150000)
    
    candidates = []
    
    # Strategy 1: Extend from previous solution
    if prev is not None and len(prev) == n - 1:
        for att in range(12):
            ext = np.zeros((n, 3))
            ext[:n-1] = prev.copy()
            
            # Try different positions
            r = compute_side(base, prev) * 0.6
            a = att * 30 * np.pi / 180
            ext[n-1] = [r * np.cos(a), r * np.sin(a), att * 30]
            candidates.append(ext)
    
    # Strategy 2: Multiple generators
    for gen in [gen_tight_spiral, gen_hex_tight, gen_rings, gen_interlocking]:
        for scale in [0.9, 1.0, 1.1]:
            init = gen(n)
            init[:, 0] *= scale
            init[:, 1] *= scale
            candidates.append(init)
    
    # Process each candidate
    for init in candidates:
        # Repair initial overlaps
        pl = repair_physics(base, init, 500, eps)
        
        # Multi-temperature SA
        for t_init in [3.0, 1.5, 0.8]:
            opt = sa_optimize_ultra(base, pl, iters // 3, t_init, 0.0001, eps)
            
            # Repair any remaining overlaps
            opt = repair_physics(base, opt, 500, eps)
            opt = repair_greedy(base, opt, 300, eps)
            
            # Compact
            opt = compact_aggressive(base, opt, 300, eps)
            opt = compact_to_bounds(base, opt, 200, eps)
            
            # Optimize rotations
            opt = optimize_rotations(base, opt, 24, eps)
            
            # Final compact
            opt = compact_aggressive(base, opt, 100, eps)
            
            # Check validity
            if not any_collision(base, opt, eps):
                s = compute_side(base, opt)
                if s < best_side:
                    best_side = s
                    best_pl = center_pl(base, opt)
    
    # Fallback if nothing worked
    if best_pl is None or any_collision(base, best_pl, eps):
        cols = max(1, int(np.ceil(np.sqrt(n))))
        pl = np.zeros((n, 3))
        for i in range(n):
            pl[i] = [(i % cols) * 1.2, (i // cols) * 1.4, 0]
        pl[:, 0] -= np.mean(pl[:, 0])
        pl[:, 1] -= np.mean(pl[:, 1])
        
        pl = repair_physics(base, pl, 2000, eps)
        best_pl = pl
        best_side = compute_side(base, best_pl)
    
    return best_pl, best_side

print("Solver ready!")

In [ ]:
# ============================================================
# SOLVE ALL PUZZLES
# ============================================================

MAX_N = 200

print("="*60)
print("🎄 Santa 2025 - Maximum Density Packing")
print("="*60)

solutions = {}
sides = {}
prev = None

start = time.time()

# Warm up JIT
print("Warming up JIT compilation...")
_ = solve_puzzle(5, TREE_POLYGON, None)
print("Ready!\n")

pbar = tqdm(range(1, MAX_N + 1), desc="Solving", unit="puzzle")

for n in pbar:
    t0 = time.time()
    pl, side = solve_puzzle(n, TREE_POLYGON, prev)
    
    # Extra validation and repair if needed
    if any_collision(TREE_POLYGON, pl, 1e-9):
        pl = repair_physics(TREE_POLYGON, pl, 2000, 1e-9)
        pl = repair_greedy(TREE_POLYGON, pl, 1000, 1e-9)
        side = compute_side(TREE_POLYGON, pl)
    
    solutions[n] = pl
    sides[n] = side
    prev = pl
    
    elapsed = time.time() - t0
    pbar.set_postfix({'n': n, 'side': f'{side:.3f}', 'time': f'{elapsed:.1f}s'})

total_time = time.time() - start
print(f"\n✅ Completed in {total_time:.1f}s ({total_time/60:.1f} min)")

In [ ]:
# ============================================================
# FINAL VALIDATION & REPAIR
# ============================================================

print("="*60)
print("🔍 Final Validation")
print("="*60)

failed = []
for n in tqdm(range(1, MAX_N + 1), desc="Validating"):
    if n not in solutions:
        failed.append((n, "Missing"))
    elif len(solutions[n]) != n:
        failed.append((n, f"Count: {len(solutions[n])}"))
    elif any_collision(TREE_POLYGON, solutions[n], 1e-9):
        failed.append((n, "Overlap"))

if failed:
    print(f"\n⚠️ {len(failed)} issues found, repairing...")
    for n, reason in tqdm(failed, desc="Repairing"):
        # Aggressive multi-phase repair
        pl = solutions[n]
        
        # Phase 1: Physics repair
        pl = repair_physics(TREE_POLYGON, pl, 3000, 1e-9)
        
        # Phase 2: Greedy repair
        pl = repair_greedy(TREE_POLYGON, pl, 2000, 1e-9)
        
        # Phase 3: If still failing, scale up and retry
        if any_collision(TREE_POLYGON, pl, 1e-9):
            pl[:, 0] *= 1.5
            pl[:, 1] *= 1.5
            pl = repair_physics(TREE_POLYGON, pl, 2000, 1e-9)
            pl = compact_aggressive(TREE_POLYGON, pl, 500, 1e-9)
        
        solutions[n] = center_pl(TREE_POLYGON, pl)
        sides[n] = compute_side(TREE_POLYGON, solutions[n])
    
    still_failed = [n for n, _ in failed if any_collision(TREE_POLYGON, solutions[n], 1e-9)]
    if still_failed:
        print(f"❌ Still failed: {still_failed}")
    else:
        print("✅ All issues fixed!")
else:
    print("\n✅ ALL PUZZLES VALID - ZERO OVERLAPS")

In [ ]:
# ============================================================
# STATISTICS
# ============================================================

print("="*60)
print("📊 Results")
print("="*60)

vals = [sides[n] for n in range(1, MAX_N + 1)]
print(f"Side length range: {min(vals):.4f} - {max(vals):.4f}")
print(f"Mean side length: {np.mean(vals):.4f}")

score = sum(sides[n]**2 / n for n in range(1, MAX_N + 1))
print(f"\n🎯 ESTIMATED SCORE: {score:.4f}")

# Check overlap status
overlaps = sum(1 for n in range(1, MAX_N + 1) if any_collision(TREE_POLYGON, solutions[n], 1e-9))
print(f"\n🔒 Puzzles with overlaps: {overlaps}/{MAX_N}")

print("\nWorst puzzles (by side length):")
for n, s in sorted(sides.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"  n={n}: side={s:.4f}, score_contrib={s**2/n:.4f}")

print("\nBest puzzles (by side length):")
for n, s in sorted(sides.items(), key=lambda x: x[1])[:5]:
    print(f"  n={n}: side={s:.4f}")

In [ ]:
# ============================================================
# CREATE SUBMISSION
# ============================================================

print("Creating submission file...")

rows = []
for n in range(1, MAX_N + 1):
    for i, p in enumerate(solutions[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{p[0]:.6f}',
            'y': f's{p[1]:.6f}',
            'deg': f's{p[2]:.6f}',
        })

submission = pd.DataFrame(rows)
print(f"Shape: {submission.shape}")
print(f"Expected: ({sum(range(1, MAX_N+1))}, 4)")
print(submission.head(10))

In [ ]:
# ============================================================
# SAVE SUBMISSION
# ============================================================

valid = all(not any_collision(TREE_POLYGON, solutions[n], 1e-9) for n in range(1, MAX_N + 1))

if valid:
    submission.to_csv('submission.csv', index=False)
    print("✅ submission.csv saved!")
    print(f"Size: {os.path.getsize('submission.csv') / 1024:.1f} KB")
else:
    print("❌ Validation failed - some puzzles have overlaps!")
    submission.to_csv('submission.csv', index=False)
    print("Saved anyway for debugging.")

print("\n" + "="*60)
print("🎄 FINAL SUMMARY")
print("="*60)
print(f"Total puzzles: {len(solutions)}/{MAX_N}")
print(f"All valid (no overlaps): {valid}")
print(f"Total time: {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"Final score: {score:.4f}")
print("="*60)